<div class="alert alert-success">
    <h1 align="center">Lesson 5: Text Classification - Sentiment Analysis</h1>
    <h3 align="center">Javad Mohammadzadeh
    </h3>
</div>

## Introduction

<h6>Text Classification:</h6> Assigning a label (from a set of pre-defined labels) to a given text.

<h6>Some applications:</h6>
- **Sentiment Analysis:** Determining the polarity of a text (`positive` or `negative`).
- **Spam detection:** email, web pages, etc.
- **Toxic text detection:** insult, racism, hatred, etc.

## Libraries

In [64]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import re
import sys
import spacy
import pickle
import numpy as np

from glob import glob
from tqdm import tqdm_notebook

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# sys.path.append(r'C:\Users\Student\Desktop\prg')
# print(sys.path)

from utils import *
from data_utils import Vocabulary, tokenizer
from train_utils import train

# setup
use_gpu = torch.cuda.is_available()
NLP = spacy.load('en_core_web_sm')  # NLP toolkit

## Tokenizing

In [65]:
text = """
Bromwell High is a cartoon comedy. 
It ran at the same time as some other programs about school life, such as 'Teachers'. 
My 35 years in the teaching profession lead me to believe that Bromwell High's 
satire is much closer to reality than is 'Teachers'. 
The scramble to survive financially, the insightful students who can see 
right through their pathetic teachers' pomp, the pettiness of the whole situation, 
all remind me of the schools I knew and their students. 
When I saw the episode in which a student repeatedly tried to burn down the school, 
I immediately recalled ......... at .......... High. 
A classic line: INSPECTOR: I'm here to sack one of your teachers. 
STUDENT: Welcome to Bromwell High. 
I expect that many adults of my age think that Bromwell High is far fetched. 
What a pity that it isn't!!!
"""

In [67]:
text = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’;]", " ", str(text))
print(text)

 Bromwell High is a cartoon comedy.  It ran at the same time as some other programs about school life, such as 'Teachers'.  My 35 years in the teaching profession lead me to believe that Bromwell High's  satire is much closer to reality than is 'Teachers'.  The scramble to survive financially, the insightful students who can see  right through their pathetic teachers' pomp, the pettiness of the whole situation,  all remind me of the schools I knew and their students.  When I saw the episode in which a student repeatedly tried to burn down the school,  I immediately recalled ......... at .......... High.  A classic line  INSPECTOR  I'm here to sack one of your teachers.  STUDENT  Welcome to Bromwell High.  I expect that many adults of my age think that Bromwell High is far fetched.  What a pity that it isn't!!! 


In [69]:
text = re.sub(r"[ ]+", " ", text)
print(text)

 Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as 'Teachers'. My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is 'Teachers'. The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line INSPECTOR I'm here to sack one of your teachers. STUDENT Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!!! 


In [70]:
text = re.sub(r"\!+", "!", text)
print(text)

 Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as 'Teachers'. My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is 'Teachers'. The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line INSPECTOR I'm here to sack one of your teachers. STUDENT Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't! 


In [15]:
tokens = [w.text for w in NLP.tokenizer(text)]
print(tokens)

[' ', 'Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', "'", 'Teachers', "'", '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', "'", 'Teachers', "'", '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'line'

### Tokenizer and Vocabulary

We have defined a function in `utils.py`, which gets the inputs text and split it to a sequence of tokens. We have used **SpaCy** toolkit for tokeniztion and you need to install it to run the codes.

```python
def tokenizer(text):
    text = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’;]", " ", str(text))
    text = re.sub(r"[ ]+", " ", text)
    text = re.sub(r"\!+", "!", text)
    text = re.sub(r"\,+", ",", text)
    text = re.sub(r"\?+", "?", text)
    return [x.text for x in NLP.tokenizer(text) if x.text != " "]
```

### Install SpaCy 

Installation:
<pre>conda install -c conda-forge spacy</pre>

Download a language:
<pre>python -m spacy download en</pre>

Usage:
```python
import spacy
NLP = spacy.load('en')
```

## Data

[IMDB](http://ai.stanford.edu/~amaas/data/sentiment/) Dataset
- A dataset for binary sentiment classification.
- It provides a set of 25,000 highly polar movie reviews for training, and 25,000 for testing

In [73]:
data_dir = r'C:\Users\Student\Desktop\prg\aclImdb'

vocab_path = 'vocab.pkl'

# parameters
max_len = 200
min_count = 10
batch_size = 50

In [74]:
os.listdir(data_dir)

['test', 'train']

In [75]:
os.listdir(f'{data_dir}/train')

['neg', 'pos']

### Statistics

In [76]:
all_filenames = glob(f'{data_dir}/*/*/*.txt')
num_words = [len(open(f).read().split(' ')) for f in tqdm_notebook(all_filenames)]

# print statistics
print('Min length =', min(num_words))
print('Max length =', max(num_words))

print('Mean = {:.2f}'.format(np.mean(num_words)))
print('Std  = {:.2f}'.format(np.std(num_words)))

print('mean + 2 * sigma = {:.2f}'.format(np.mean(num_words) + 2.0 * np.std(num_words)))

<ipython-input-76-8081751c11bd>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  num_words = [len(open(f).read().split(' ')) for f in tqdm_notebook(all_filenames)]


  0%|          | 0/50000 [00:00<?, ?it/s]

Min length = 4
Max length = 2470
Mean = 231.15
Std  = 171.32
mean + 2 * sigma = 573.80


## Dataset

In [77]:
print(num_words[:20])
PAD = '<pad>'  # special symbol we use for padding text
UNK = '<unk>'  # special symbol we use for rare or unknown word

[168, 234, 225, 308, 242, 116, 164, 140, 129, 161, 113, 339, 247, 112, 185, 213, 148, 196, 350, 78]


In [78]:
class TextClassificationDataset(Dataset):
    
    def __init__(self, path, tokenizer, 
                 split='train', 
                 vocab_path='vocab.pkl', 
                 max_len=100, min_count=10):
        
        self.path = path
        assert split in ['train', 'test']
        self.split = split
        self.vocab_path = vocab_path
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.min_count = min_count
        
        self.cache = {}
        self.vocab = None
        
        self.classes = []
        self.class_to_index = {}
        self.text_files = []
        
        split_path = f'{path}/{split}'
        for cls_idx, label in enumerate(os.listdir(split_path)):
            text_files = [(fname, cls_idx) for fname in glob(f'{split_path}/{label}/*.txt')]
            self.text_files += text_files
            self.classes += [label]
            self.class_to_index[label] = cls_idx
        
        self.num_classes = len(self.classes)
            
        # build vocabulary from training and validation texts
        self.build_vocab()
        
    def __getitem__(self, index):#[]
        # read the tokenized text file and its label (neg=0, pos=1)
        fname, class_idx = self.text_files[index]
        
        if fname in self.cache:
            return self.cache[fname], class_idx
        
        # read text file 
        text = open(fname).read()
        
        # tokenize the text file
        tokens = self.tokenizer(text.lower())
        
        # padding and trimming
        if len(tokens) < self.max_len:
            num_pads = self.max_len - len(tokens)
            tokens = [PAD] * num_pads + tokens
        elif len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
            
        # numericalizing
        ids = torch.LongTensor(self.max_len)
        for i, word in enumerate(tokens):
            if word not in self.vocab.word2index:
                ids[i] = self.vocab.word2index[UNK]  # unknown words
            elif word != PAD and self.vocab.word2count[word] < self.min_count:
                ids[i] = self.vocab.word2index[UNK]  # rare words
            else:
                ids[i] = self.vocab.word2index[word]
                
        # save in cache for future use
        self.cache[fname] = ids
        
        return ids, class_idx
    
    def __len__(self):
        return len(self.text_files)
    
    def build_vocab(self):
        if not os.path.exists(self.vocab_path):
            vocab = Vocabulary(self.tokenizer)
            filenames = glob(f'{path}/*/*/*.txt')
            for filename in tqdm(filenames, desc='Building Vocab'):
                with open(filename, encoding='utf8') as f:
                    for line in f:
                        vocab.add_sentence(line.lower())

            # sort words by their frequencies
            words = [(0, PAD), (0, UNK)]
            words += sorted([(c, w) for w, c in vocab.word2count.items()], reverse=True)

            self.vocab = Vocabulary(self.tokenizer)
            for i, (count, word) in enumerate(words):
                self.vocab.word2index[word] = i
                self.vocab.word2count[word] = count
                self.vocab.index2word[i] = word
                self.vocab.count += 1

            pickle.dump(self.vocab, open(self.vocab_path, 'wb'))
        else:
            self.vocab = pickle.load(open(self.vocab_path, 'rb'))

In [79]:
train_ds = TextClassificationDataset(data_dir, tokenizer, 'train', vocab_path, max_len, min_count)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

valid_ds = TextClassificationDataset(data_dir, tokenizer, 'test', vocab_path, max_len, min_count)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)

In [80]:
len(train_ds)

25000

In [81]:
len(valid_ds)

25000

In [82]:
train_ds.classes


['neg', 'pos']

In [83]:
train_ds.class_to_index

{'neg': 0, 'pos': 1}

In [84]:
ids, label = train_ds[3]
print(len(ids.numpy()))
print(ids.numpy())

print(train_ds.classes[label])
print(ids.numpy())

200
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0   761   299
     3    13   130    14     9   443     8    34    40   504    25     3
    24  1372     3    38   143    33  2713    54  1963    36     2  2703
    42    93    87  2297    75  3924    54     5    29   115     4   265
     2   138  1719     5 13564   473    19   326     3    14    76     9
   108  1381     8   115     4     2  4078     7     6   452  1353    19
  1783     4     2     1   645    78   108   204     4    98   204    70
    26   115   117    93    47  1233    45     5   670    59   266    60
     2   421     9   117    93  5045     4    13    73    77     6   256
    69    47   388   151    14    25     4     2   380    78   326     3
    24    98    86     7    15   458     3 15115     3  3367     3   535
    70    26   198    59     2    72   158    13   432    19  9273 16644
     5    50   815  2654     5  1093   138     

In [85]:
# convert back the sequence of integers into original text
ids=ids.numpy()
print(' '.join([train_ds.vocab.index2word[i] for i in ids]))

<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> sorry everyone , i know this is supposed to be an art film , but wow , they should have handed out guns at the screening so people could blow their brains out and not watch . although the scene design and photographic direction was excellent , this story is too painful to watch . the absence of a sound track was brutal . the <unk> shots were too long . how long can you watch two people just sitting there and talking ? especially when the dialogue is two people complaining . i really had a hard time just getting through this film . the performances were excellent , but how much of that dark , sombre , uninspired , stuff can you take ? the only thing i liked was maureen stapleton and her red dress and dancing scene . otherwise this was a ripoff of bergman . and i 'm no fan f his either . i think anyone who says they enjoyed 1 1 2 hours of this is , well , ly

In [86]:
# print the original text
print(open(train_ds.text_files[0][0]).read())

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.


### Vovcabulary size

In [87]:
vocab = train_ds.vocab
freqs = [(count, word) for (word, count) in vocab.word2count.items() if count >= min_count]
vocab_size = len(freqs) + 2  # for PAD and UNK tokens
print(f'Vocab size = {vocab_size}')

print('\nMost common words:')
for c, w in sorted(freqs, reverse=True)[:50]:
    print(f'{w}: {c}')

Vocab size = 29512

Most common words:
the: 666690
,: 543308
.: 469782
and: 324154
a: 321797
of: 289312
to: 267959
is: 217022
>: 202239
it: 187957
in: 186452
i: 173859
this: 150166
that: 143987
's: 122009
<: 119154
br: 119019
was: 99882
as: 91665
for: 87301
with: 87183
movie: 86122
but: 83310
film: 78079
you: 68703
on: 67730
n't: 66056
not: 62603
are: 60025
he: 58686
his: 57501
have: 56757
be: 53208
one: 53082
at: 46724
all: 46418
they: 45218
by: 44432
an: 42945
who: 42196
so: 40701
from: 40439
like: 40089
there: 37011
or: 35736
just: 35142
do: 34982
!: 34878
her: 34548
about: 33987


## LSTM Classifier

In [88]:
class LSTMClassifier(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, num_classes, batch_size):
        super(LSTMClassifier, self).__init__()
        
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(vocab_size, embed_size) # a lookup table
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, dropout=0.3, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(2*hidden_size, 100),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(100, num_classes)
        )
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        h = to_var(torch.zeros((2*self.num_layers, self.batch_size, self.hidden_size)))
        c = to_var(torch.zeros((2*self.num_layers, self.batch_size, self.hidden_size)))
        return h, c
    
    def forward(self, x):
        x = self.embedding(x)
        x, self.hidden = self.lstm(x, self.hidden)
        x = self.fc(x[-1])  # select the last output
        return x

In [89]:
vocab_size = 2 + len([w for (w, c) in train_ds.vocab.word2count.items() if c >= min_count])
print(vocab_size)

29512


## Model

In [90]:
# LSTM parameters
embed_size = 100
hidden_size = 256
num_layers = 1

# training parameters
lr = 0.001
num_epochs = 10

In [91]:
model = LSTMClassifier(embed_size=embed_size, 
                       hidden_size=hidden_size, 
                       vocab_size=vocab_size,
                       num_layers=num_layers,
                       num_classes=train_ds.num_classes, 
                       batch_size=batch_size)

if use_gpu:
    model = model.cuda()

In [92]:
criterion = nn.CrossEntropyLoss()
if use_gpu:
    criterion = criterion.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.7, 0.99))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.975)

In [ ]:
# model.load_state_dict(torch.load('models/lstm-3-400-10-256-1024-1-0.85764.pth'))

### Training

In [ ]:
hist = train(model, train_dl, valid_dl, criterion, optimizer, scheduler, num_epochs)

Training:   0%|          | 0/500 [00:00<?, ?it/s]

C:\Users\Student\Desktop\prg\train_utils.py:32: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), 0.3)


## Load a trained model

In [ ]:
# LSTM parameters
max_len = 400
min_count = 10
embed_size = 256
hidden_size = 1024
num_layers = 1


model = LSTMClassifier(embed_size=embed_size, 
                       hidden_size=hidden_size, 
                       vocab_size=vocab_size,
                       num_layers=num_layers,
                       num_classes=train_ds.num_classes, 
                       batch_size=batch_size)

model.load_state_dict(torch.load('models/tmp/lstm-1-400-10-256-1024-1-0.87964.pth'))

if use_gpu:
    model = model.cuda()

## Visualizing word vectors

<img src='imgs/wordvecs_persian.png' width='80%'/>

In [30]:
word_vecs = model.embedding.weight.data =  

In [31]:
print(word_vecs.size())

torch.Size([29512, 100])


In [32]:
print(word_vecs[0])


-1.0678
-0.8920
-0.0058
-1.3950
 0.8038
 0.9048
 0.1682
-0.2589
 2.0809
-1.5028
-0.1373
 0.9797
-1.3226
 0.1690
 0.1304
 1.2229
-1.5732
-0.0680
 0.2411
-0.5359
-1.1798
 1.5737
-0.0653
-1.5709
-0.1531
-1.6529
 0.9688
 0.0960
-0.0773
-0.0464
-1.1947
-0.5481
 0.2004
-2.5060
 0.3788
-1.3304
-0.2268
 0.5360
-0.7743
-1.7638
-0.7493
-0.5174
-0.4552
 0.3821
-0.7965
 0.0837
-0.1656
-0.3691
 1.0137
-0.6089
 0.8563
-0.4448
-0.5062
 0.9916
 1.4466
-0.2972
 0.3063
 0.3336
 0.1705
-1.5537
-0.1382
-0.5471
-1.1656
 0.6987
 0.7385
 1.0847
-1.3165
 1.1401
 0.0106
 0.5664
-0.6338
 0.0254
-0.0188
-0.9588
-0.4423
 0.2074
 0.3086
-0.6075
-1.9901
 0.5488
-1.5887
-1.1958
 0.6825
-0.4171
 0.7192
-0.7814
-1.2733
 0.7871
-0.2044
 0.6582
 1.0289
 0.9307
-0.0660
 0.9300
 0.0040
 0.5446
 1.5127
 0.8484
-0.9488
 0.6794
[torch.cuda.FloatTensor of size 100 (GPU 0)]



## Improvements

- Fine-tuning hyper-parameters
- Bidirectional LSTM
- Pre-trained word vectors ([GloVe](https://nlp.stanford.edu/projects/glove/), [FastText](https://code.facebook.com/posts/1438652669495149/fair-open-sources-fasttext/), etc.)
- Dropouts and regularization (https://arxiv.org/pdf/1708.02182.pdf)
- Attention mechanism (https://distill.pub/2016/augmented-rnns/)
- Use GRU or QRNN (https://arxiv.org/pdf/1611.01576.pdf)
- Pre-training with a language model (https://arxiv.org/pdf/1605.07725.pdf)

### Attention mechanism

<img src='imgs/attention-example.png' width='90%'/>

### QRNN

<img src='imgs/QRNN.png' width='100%'/>

## Programming Excersize: Toxic Texts

Try to get an accuracy equal (or above 90) % for IMDB dataset.

## Further Study

<h6>LSTM</h6> 
- http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714

<h6>Word Vectors and NLP</h6>
- https://einstein.ai/research/learned-in-translation-contextualized-word-vectors